## 7.4 Priority queues

Families with young children often board an airplane before other passengers.
Hospitals treat patients with more severe conditions first.
To-do list apps list tasks with higher priority first.

All these are examples of queues that don't process items in a FIFO order.
A **priority queue** is a sequence in which each item has a priority and
items are removed from the queue from the highest to the lowest priority.
In a **max-priority queue**, the highest priority corresponds to the largest
value, whereas in a **min-priority queue** it is given by the smallest value.
For example, if priorities are represented by positive integers, priority 1 is
the highest priority in a min-priority queue and
the lowest priority in a max-priority queue.

In the rest of this section we only implement unbounded max-priority queues,
as bounded and min-priority queues are implemented very similarly.
Actually, if priorities are given by integers,
we get a max-priority queue to behave like a min-priority queue
by negating the priorities when adding the items to the queue.
Suppose item A has priority 1 and item B has priority&nbsp;3.
By inserting them with priorities −1 and −3, the max-priority queue
will return A before B, as if it were a min-priority queue.

Usually the order among items with the same priority is arbitrary,
but a fairer priority queue keeps them in FIFO order.

Traditionally, priority queue operations are named differently
from queue operations, to make clear which kind of queue we're using.

Operation | Effect | Algorithm in English
:-|:-|:-
new | create an empty priority queue | let *pq* be a new priority queue
length | the number of items in *pq* |  │*pq*│
insert  | add item *i* with priority *p* to *pq*  |  add (*p*, *i*) to *pq*
find max | obtain an item with highest-priority value  | max(*pq*)
remove max | remove the item obtained by find max   | remove max(*pq*)

The find and remove operations are only defined for non-empty priority queues.

### 7.4.1 With dynamic arrays: version 1

One simple way of implementing an unbounded max-priority queue is
to use a dynamic array of priority–item pairs
that are kept in ascending priority order.
The insert operation appends a new priority–item pair and re-sorts the array.
The find and remove operations simply access and remove the last pair.

#### Exercise 7.4.1

With dynamic arrays, the new and length operations take constant time.
What is the complexity of the other operations for the approach outlined?

Operation | Complexity
:-|:-
insert  |
find max  |
remove max  |

[Hint](../31_Hints/Hints_07_4_01.ipynb)
[Answer](../32_Answers/Answers_07_4_01.ipynb)

In Python, we can use a list of tuples to represent the priority queue.
Here's a simple example, with priorities from 1 (lowest) to 3 (highest).

In [1]:
tasks = []  # a priority queue
tasks.append(("go on holiday", 2))
tasks.sort()
tasks.append(("finish this chapter", 3))
tasks.sort()
tasks.append(("answer email", 1))
tasks.sort()
for times in range(len(tasks)):  # noqa: B007
    print(tasks[-1][1], tasks[-1][0])  # print priority and task
    tasks.pop(-1)

2 go on holiday
3 finish this chapter
1 answer email


This is not quite working. The task with priority 3&nbsp;should be listed first.
Where's the error?

___

The tuples are put in ascending order by comparing them item by item
(lexicographic order): first the task, then its priority.
So the tasks end up in alphabetical order in the list and
the task starting with 'g' is considered the highest-priority task.

How can we fix the issue?

___

Since the ordering is by priority, that must be the first item of the pair.

Here's a similar example to show the correct ordering.

In [2]:
tasks = []  # a priority queue
tasks.append((2, "go on holiday"))
tasks.append((1, "answer email"))
tasks.append((3, "finish this chapter"))
tasks.append((1, "remove old files"))
tasks.sort()
while tasks != []:
    task = tasks.pop(-1)
    print(task[0], task[1])

3 finish this chapter
2 go on holiday
1 remove old files
1 answer email


This example shows both the advantages and disadvantages of using Python lists
instead of defining our own class. The advantages are a familiar notation
and the flexibility of using list operations:
we can sort only when needed,
we can access and remove a highest-priority item with one operation,
we can remove any other item if they leave the queue, etc.

One disadvantage is that this approach exposes the data structure and
doesn't restrict the operations, which can lead to mistakes.
Another disadvantage is that, by sorting the tuples, the approach assumes
that the items in the queue are comparable, thus restricting its applicability.

<div class="alert alert-warning">
<strong>Note:</strong> Try to restrict data types as little as possible.
</div>

Sorting priority–item pairs also leads to the priority queue not being fair.
Among items with the same priority, the first to be removed is the one with
highest-item value, not the one waiting longest.
In the example, answering emails was added before but is done after the other
priority 1 task, because of the alphabetical ordering.

### 7.4.2 With dynamic arrays: version 2

The solution to all these issues is to define our own class,
with a bespoke insertion algorithm that is fair and only compares priorities.

<div class="alert alert-warning">
<strong>Note:</strong> Using bespoke algorithms instead of general-purpose ones
often leads to a better solution.
</div>

Here's an incomplete solution with separate lists (dynamic arrays) for
the priorities and the items.

In [3]:
# this code is also in m269_priority.py


class ArrayPriorityQueue:
    """A dynamic array implementation of a fair max-priority queue.

    Items with the same priority are retrieved in FIFO order.
    """

    def __init__(self) -> None:
        """Create a new empty priority queue."""
        self.priorities = []  # in ascending order
        self.items = []

    def length(self) -> int:
        """Return the number of items in the queue."""
        return len(self.items)

    def find_max(self) -> object:
        """Return the oldest item with the highest priority.

        Preconditions: self.length() > 0
        """
        return self.items[-1]

    def remove_max(self) -> None:
        """Remove the oldest item with the highest priority.

        Preconditions: self.length() > 0
        """
        self.items.pop(-1)
        self.priorities.pop(-1)

    def insert(self, item: object, priority: object) -> None:
        """Add item with the given priority to the queue.

        Preconditions:
        - priority is comparable to the priorities of all existing items
        """
        index = 0
        # compute the index where to insert the item
        self.items.insert(index, item)
        self.priorities.insert(index, priority)

#### Exercise 7.4.2

Write an algorithm in English that finds the index of where to insert the item.
Use variables *priorities* and *priority* and set a value for *index*.
You can assume that priorities can be compared with <, ≤, ≠, etc.

_Write your answer here._

[Hint](../31_Hints/Hints_07_4_02.ipynb)
[Answer](../32_Answers/Answers_07_4_02.ipynb)

#### Exercise 7.4.3

Implement the algorithm in method `insert`. Run the following test,
where `None` represents a patient without identification
to make sure that the items are not comparable.

In [4]:
%run -i ../m269_test

hospital = ArrayPriorityQueue()
hospital.insert("Bob", 1)
hospital.insert("Alice", 3)
hospital.insert(None, 1)
for patient in ("Alice", "Bob", None):  # this is the expected order
    check("find_max", hospital.find_max(), patient, hospital.length())
    hospital.remove_max()

[Answer](../32_Answers/Answers_07_4_03.ipynb)

#### Exercise 7.4.4

Fill out the following table for the described approach.

Operation | Complexity
:-|:-
insert  |
find max  |
remove max  |

[Hint](../31_Hints/Hints_07_4_04.ipynb)
[Answer](../32_Answers/Answers_07_4_04.ipynb)

#### Exercise 7.4.5

Is it more efficient to have two lists for the items and the priorities,
a single list with priority–item pairs, or does it not matter much?
By more efficient I mean doing less work,
not necessarily having better complexity.

_Write your answer here._

[Hint](../31_Hints/Hints_07_4_05.ipynb)
[Answer](../32_Answers/Answers_07_4_05.ipynb)

#### Exercise 7.4.6

There's yet another way of implementing a priority queue with dynamic arrays,
with the following complexities:

Operation | Complexity
:-|:-
insert  |  Θ(1)
find max  |  Θ(│*pq*│)
remove max  |  Θ(│*pq*│)

How? Outline the algorithm for each operation.

[Answer](../32_Answers/Answers_07_4_06.ipynb)

### 7.4.3 With a linked list

Version 2 of the dynamic array implementation can be adapted to linked
lists. The head node has the item with the highest-priority value, i.e.
the item returned by find max.
To insert a new item, we start from the head node,
skip all nodes with higher or equal priority, and insert a new node in the found place.
With a linked list, no items are copied when inserting a new one:
the insertion itself takes constant time once the insertion point is found,
and there's no need to resize a linked list.

### 7.4.4 Min-priority queues

To finish this section, here's an example of using a max-priority queue
as a min-priority queue.

Calendar apps allow us to attach a reminder to an event with a period
of our choice, e.g. 15&nbsp;minutes before the event.
Reminders can be implemented as a priority queue, where an item is an
event and its priority is the time when to issue the reminder.
For example, the priority of a 3&nbsp;pm meeting with a 30-minute reminder
is 2:30&nbsp;pm. The reminders are issued from earliest to latest time,
so it's a min-priority queue, where the priority is the time.

Times can be represented as integers (minutes after midnight),
so we can negate them to use our max-priority queue implementation.
Here's an example, using an already fully implemented class.

In [5]:
%run -i ../m269_priority

reminders = ArrayPriorityQueue()
# meeting with 30 minute advance reminder
reminders.insert("research group meeting @ 3pm", -(14 * 60 + 30))
# meetings with same reminder time
reminders.insert("M269 team meeting @ 11am", -(9 * 60))
reminders.insert("student supervision @ 10am", -(9 * 60))
while reminders.length() > 0:
    print(reminders.find_max())
    reminders.remove_max()

M269 team meeting @ 11am
student supervision @ 10am
research group meeting @ 3pm


#### Exercise 7.4.7

Why not model reminders with a FIFO queue?

_Write your answer here._

[Hint](../31_Hints/Hints_07_4_07.ipynb)
[Answer](../32_Answers/Answers_07_4_07.ipynb)

#### Exercise 7.4.8

The workaround for a max-priority queue to behave like a min-priority queue
only works for integer priorities. Briefly explain how you would change our
class's code to implement a min-priority queue for any comparable
priority values.

_Write your answer here._

[Hint](../31_Hints/Hints_07_4_08.ipynb)
[Answer](../32_Answers/Answers_07_4_08.ipynb)

⟵ [Previous section](07_3_queue.ipynb) | [Up](07-introduction.ipynb) | [Next section](07_5_summary.ipynb) ⟶